<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/tensorflow/Hyperparameter_Optimization_in_TensorFlow_using_W%26B_Sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{sweeps-keras} -->

# 🧹 Introduction to Hyperparameter Sweeps using W&B and Keras

Searching through high dimensional hyperparameter spaces to find the most performant model can get unwieldy very fast. Hyperparameter sweeps provide an organized and efficient way to conduct a battle royale of models and pick the most accurate model. They enable this by automatically searching through combinations of hyperparameter values (e.g. learning rate, batch size, number of hidden layers, optimizer type) to find the most optimal values.

In this tutorial we'll see how you can run sophisticated hyperparameter sweeps in 3 easy steps using Weights and Biases.

![](https://i.imgur.com/WVKkMWw.png)

## Sweeps: An Overview

Running a hyperparameter sweep with Weights & Biases is very easy. There are just 3 simple steps:

1. **Define the sweep:** we do this by creating a dictionary or a [YAML file](https://docs.wandb.com/library/sweeps/configuration) that specifies the parameters to search through, the search strategy, the optimization metric et all.

2. **Initialize the sweep:** with one line of code we initialize the sweep and pass in the dictionary of sweep configurations:
`sweep_id = wandb.sweep(sweep_config)`

3. **Run the sweep agent:** also accomplished with one line of code, we call `wandb.agent()` and pass the `sweep_id` to run, along with a function that defines your model architecture and trains it:
`wandb.agent(sweep_id, function=train)`

And voila! That's all there is to running a hyperparameter sweep! In the notebook below, we'll walk through these 3 steps in more detail.


We highly encourage you to fork this notebook so you can tweak the parameters,
try out different models,
or try a Sweep with your own dataset!

## Resources
- [Sweeps docs →](https://docs.wandb.ai/sweeps)
- [Launching from the command line →](https://www.wandb.com/articles/hyperparameter-tuning-as-easy-as-1-2-3)


# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [1]:
!pip install -Uqqq wandb

### Step 1️⃣: Import W&B and Login

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: tcapelle (use `wandb login --relogin` to force relogin)


True

> Side note: If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login()` will take you to sign-up/login page. Signing up is as easy as a few clicks.

## 👩‍🍳 Prepare Dataset

We will use MNIST directly from `keras.datasets`

In [4]:
# Get the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
num_classes = len(np.unique(y_train))
input_shape = x_train.shape[-2:] + (1,)

prepare data for training:
- scale in [0,1]
- transform targets to cateforical

In [5]:
# Scale
x_train = x_train / 255.
x_test = x_test / 255.

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 🧠 Define the Model and Training Loop

## 🏗️ Build a Simple Classifier

In [6]:
def ConvNet(dropout=0.2):
    return keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(dropout),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model = ConvNet()

model.summary()

Metal device set to: Apple M1 Pro
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)         

2021-12-03 14:50:31.916472: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-03 14:50:31.916592: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 3️⃣ Write a Training scrip

In [17]:
def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)

def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):  
    
    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(loss="categorical_crossentropy", 
                  optimizer=get_optimizer(lr, optimizer), 
                  metrics=["accuracy"])

    # callback setup
    cbs = [wandb.keras.WandbCallback(log_batch_frequency=log_freq)]

    model.fit(x_train, 
              y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(x_test, y_test), 
              callbacks=cbs)

# 4️⃣ Define the Sweep

Fundamentally, a Sweep combines a strategy for trying out a bunch of hyperparameter values with the code that evalutes them.
Whether that strategy is as simple as trying every option
or as complex as [BOHB](https://arxiv.org/abs/1807.01774),
Weights & Biases Sweeps have you covered.
You just need to _define your strategy_
in the form of a [configuration](https://docs.wandb.com/sweeps/configuration).

When you're setting up a Sweep in a notebook like this,
that config object is a nested dictionary.
When you run a Sweep via the command line,
the config object is a
[YAML file](https://docs.wandb.com/sweeps/quickstart#2-sweep-config).

Let's walk through the definition of a Sweep config together.
We'll do it slowly, so we get a chance to explain each component.
In a typical Sweep pipeline,
this step would be done in a single assignment.

### 👈 Pick a `method`

The first thing we need to define is the `method`
for choosing new parameter values.

We provide the following search `methods`:
*   **`grid` Search** – Iterate over every combination of hyperparameter values.
Very effective, but can be computationally costly.
*   **`random` Search** – Select each new combination at random according to provided `distribution`s. Surprisingly effective!
*   **`bayes`ian Search** – Create a probabilistic model of metric score as a function of the hyperparameters, and choose parameters with high probability of improving the metric. Works well for small numbers of continuous parameters but scales poorly.

We'll stick with `random`.

In [8]:
sweep_config = {
    'method': 'bayes'
    }

For `bayes`ian Sweeps,
you also need to tell us a bit about your `metric`.
We need to know its `name`, so we can find it in the model outputs
and we need to know whether your `goal` is to `minimize` it
(e.g. if it's the squared error)
or to `maximize` it
(e.g. if it's the accuracy).

In [9]:
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

If you're not running a `bayes`ian Sweep, you don't have to,
but it's not a bad idea to include this in your `sweep_config` anyway,
in case you change your mind later.
It's also good reproducibility practice to keep note of things like this,
in case you, or someone else,
come back to your Sweep in 6 months or 6 years
and don't know whether `val_G_batch` is supposed to be high or low.

### 📃 Name the hyper`parameters`

Once you've picked a `method` to try out new values of the hyperparameters,
you need to define what those `parameters` are.

Most of the time, this step is straightforward:
you just give the `parameter` a name
and specify a list of legal `values`
of the parameter.

For example, when we choose the `optimizer` for our network,
there's only a finite number of options.
Here we stick with the two most popular choices, `adam` and `sgd`.
Even for hyperparameters that have potentially infinite options,
it usually only makes sense to try out
a few select `values`,
as we do here with `dropout`.

In [10]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.1, 0.3, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

It's often the case that there are hyperparameters
that we don't want to vary in this Sweep,
but which we still want to set in our `sweep_config`.

In that case, we just set the `value` directly:

In [11]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

For a `grid` search, that's all you ever need.

For a `random` search,
all the `values` of a parameter are equally likely to be chosen on a given run.

If that just won't do,
you can instead specify a named `distribution`,
plus its parameters, like the mean `mu`
and standard deviation `sigma` of a `normal` distribution.

See more on how to set the distributions of your random variables [here](https://docs.wandb.com/sweeps/configuration#distributions).

In [12]:
import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.001,
        'max': 0.1
      },
    'batch_size': {
        'values': [64, 128]
      }
    })

When we're finished, `sweep_config` is a nested dictionary
that specifies exactly which `parameters` we're interested in trying
and what `method` we're going to use to try them.

In [13]:
import pprint

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': [64, 128]},
                'dropout': {'values': [0.1, 0.3, 0.5]},
                'epochs': {'value': 1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.001},
                'optimizer': {'values': ['adam', 'sgd']}}}


But that's not all of the configuration options!

For example, we also offer the option to `early_terminate` your runs with the [HyperBand](https://arxiv.org/pdf/1603.06560.pdf) scheduling algorithm. See more [here](https://docs.wandb.com/sweeps/configuration#stopping-criteria).

You can find a list of all configuration options [here](https://docs.wandb.com/library/sweeps/configuration)
and a big collection of examples in YAML format [here](https://github.com/wandb/examples/tree/master/examples/keras/keras-cnn-fashion).



# Step 5️⃣: Wrap the Training Loop

You'll need a function, like `sweep_train` below,
that uses `wandb.config` to set the hyperparameters
before `train` gets called.

In [14]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "ConvNet"
        wandb.config.dataset_name = "MNIST"

        # initialize model
        model = ConvNet(wandb.config.dropout)

        train(model, 
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

# Step 6️⃣: Initialize Sweep and Run Agent 

In [15]:
sweep_id = wandb.sweep(sweep_config, project="sweeps-keras")

Create sweep with ID: 5b7n5zpy
Sweep URL: https://wandb.ai/tcapelle/sweeps-keras/sweeps/5b7n5zpy


In [16]:
wandb.agent(sweep_id, function=sweep_train, count=30)

wandb: Agent Starting Run: 46gvfni6 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	epochs: 1
wandb: 	learning_rate: 0.09970648423369136
wandb: 	optimizer: sgd


2021-12-03 14:50:38.105927: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-03 14:50:38.106110: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-03 14:50:38.216588: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 0.3477 - accuracy: 0.8947

2021-12-03 14:50:43.722534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 6s 13ms/step - loss: 0.3477 - accuracy: 0.8947 - val_loss: 0.1094 - val_accuracy: 0.9675


...

2021-12-03 15:01:07.952456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - ETA: 0s - loss: 0.3973 - accuracy: 0.8752

2021-12-03 15:01:18.405615: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 12s 12ms/step - loss: 0.3973 - accuracy: 0.8752 - val_loss: 0.1255 - val_accuracy: 0.9639


accuracy,▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
epoch,▁
loss,██▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁
val_loss,▁
accuracy,0.87523
best_epoch,0
best_val_loss,0.12546
epoch,0
loss,0.39727
val_accuracy,0.9639


# 👀 Visualize Results

Click on the **Sweep URL** link above to see your live results.


# 🎨 Example Gallery

See examples of projects tracked and visualized with W&B in our [Gallery →](https://app.wandb.ai/gallery)

# 📏 Best Practices
1. **Projects**: Log multiple runs to a project to compare them. `wandb.init(project="project-name")`
2. **Groups**: For multiple processes or cross validation folds, log each process as a runs and group them together. `wandb.init(group='experiment-1')`
3. **Tags**: Add tags to track your current baseline or production model.
4. **Notes**: Type notes in the table to track the changes between runs.
5. **Reports**: Take quick notes on progress to share with colleagues and make dashboards and snapshots of your ML projects.

# 🤓 Advanced Setup
1. [Environment variables](https://docs.wandb.com/library/environment-variables): Set API keys in environment variables so you can run training on a managed cluster.
2. [Offline mode](https://docs.wandb.com/library/technical-faq#can-i-run-wandb-offline): Use `dryrun` mode to train offline and sync results later.
3. [On-prem](https://docs.wandb.com/self-hosted): Install W&B in a private cloud or air-gapped servers in your own infrastructure. We have local installations for everyone from academics to enterprise teams.